# Visualise Sites

In [1]:
import pdal
import pandas as pd
import numpy as np
import geopandas as gpd

import pyvista as pv
pv.set_jupyter_backend('client')

from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

In [2]:
site_lidar_dir = Path('../data/outputs/sites/lidar')
sites_gdf = gpd.read_file('../data/outputs/sites/sites.geojson')
sites = sites_gdf['site'].to_list()

In [3]:
def read_point_cloud(file_path):
    pipeline = pdal.Reader(file_path) | pdal.Filter('filters.range', limits='Classification[0:5]')
    pipeline.execute()
    return pd.DataFrame(pipeline.arrays[0])

def plot_point_cloud(df, color='rgb'):
    X = df["X"] - df["X"].min()
    Y = df["Y"] - df["Y"].min()
    Z = df["Z"]
    positions = np.column_stack((X, Y, Z)).astype("float32")

    R = (df["Red"] / 65535 * 255).astype("uint8")
    G = (df["Green"] / 65535 * 255).astype("uint8")
    B = (df["Blue"] / 65535 * 255).astype("uint8")
    colors = np.column_stack((R, G, B))

    mesh = pv.PolyData(positions)
    mesh['height'] = Z
    mesh['rgb'] = colors
    mesh.plot(point_size=2, scalars=color, rgb=(color == 'rgb'), cpos='xz', show_axes=True, notebook=True)

def read_and_plot_site(site: str, color = 'rgb'):
    file_path = str(site_lidar_dir / f"{site}.copc.laz")
    lidar_df = read_point_cloud(file_path)
    plot_point_cloud(lidar_df, color=color)

In [ ]:
# Create interactive widgets for site selection
site_dropdown = widgets.Dropdown(
    options=sites,
    value=sites[0] if sites else None,
    description='Site:',
    disabled=False,
)

color_dropdown = widgets.Dropdown(
    options=['rgb', 'height'],
    value='rgb',
    description='Color by:',
    disabled=False,
)

# Function to handle the plot update
def update_plot(site, color):
    read_and_plot_site(site, color=color)

# Create interactive plot
interactive_plot = widgets.interactive(update_plot, site=site_dropdown, color=color_dropdown)
display(interactive_plot)

interactive(children=(Dropdown(description='Site:', options=('AGG_O_01', 'AGG_O_05', 'AGG_O_07', 'AGG_Y_02', '…